<a href="https://colab.research.google.com/github/kartk/ML/blob/main/Decision_Trees_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git init


Initialized empty Git repository in /content/.git/


In [3]:
!git config --global user.email "kaarthikgovindappa@gmail.com"

In [4]:
!git config --global user.name "kartk"

In [5]:
!ls

sample_data


In [6]:
!pwd

/content
